In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def rates(url, currency):
    try:
        html = urlopen(url)
    except:
        print('Invalid url, insert new one:')
        html = urlopen(str(input()))
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find('table', {'class':'tablestats', 'id':'dataTableID'})
    rows = soup.find_all('tr')

    table_list = []

    # For every row in the table, find each cell element and add it to the list
    for row in rows:
        row_td = row.find_all('td')
        row_cells = str(row_td)
        row_cleantext = BeautifulSoup(row_cells, "lxml").get_text()  # extract the text without HTML tags
        table_list.append(row_cleantext)  # Add the clean table row to the list

    #table_list_clean = table_list[30:]
    if currency=='pound' or currency=='real':
        table_list_clean = table_list[31:]
    elif currency=='dollar':
        table_list_clean = table_list[30:]
        
    df = pd.DataFrame(table_list_clean)
    df = df[0].str.split(',', expand=True)
    df = df.drop(2,axis=1)
    df[0] = df[0].str.strip('[')
    df[0] = df[0].str.strip(']')
    try:
        df[0].iloc[0] = df[0].iloc[0].strip('↓')
    except:
        df[0].iloc[0] = df[0].iloc[0].strip('↑')
        
    df.columns = df.iloc[0]
    df = df.drop(0, axis=0)
    df=df.sort_values(by=[df.columns[0]])
    
    return df

In [3]:
def main(_):
    
    print('Insert currency (pound, dollar or real):')
    currency = str(input())
    print('Insert url:')
    dollar_rates = rates(str(input()), currency)
    print('Write current file name, including extension (write None if there is no current file):')
    file_name = str(input())
    
    if file_name=='None':
        print('Write destination file name:')
        dest_file = str(input())
        return dollar_rates.to_csv(dest_file+'.csv', index=False)
    else:
        try:
            my_file = pd.read_csv(file_name)
        except:
            print('File not found. Please write the file name again, including extension:')
            file_name = str(input())
            my_file = pd.read_csv(file_name)
        if my_file.equals(dollar_rates):
            return print('No new data found. Exiting.')
        else:
            print('New data found. Saving changes.')
            return dollar_rates.to_csv('/home/diogo_mota/Desktop/dollar_rates.csv', index=False)

In [5]:
main()
print('Done.')

Write current file name, including extension (write None if there is no current file):
None
Insert currency (pound, dollar or real):
dollar
Insert url:
https://sdw.ecb.europa.eu/quickview.do?SERIES_KEY=120.EXR.D.USD.EUR.SP00.A&
Write destination file name:
dolalr_rate
Done.
